In [7]:
from glob import glob

image_path_list = glob("images/*")
image_path_list

['images/15_31 30124013_10213858372666214_6084300428709724160_n.jpg',
 'images/5_7 46741305_10215499670297629_8916867902003478528_n.jpg',
 'images/11_0 83630985_10218853940432286_7213858926134034432_n.jpg',
 'images/NA 453632882_10229805741060457_7489649556786205200_n.jpg',
 'images/21_5 60337780_10216655216265556_1011334716533506048_n.jpg',
 'images/18_54 405741160_10228415558266756_3898063265316050459_n.jpg',
 'images/30_0 48362178_10215595765219942_1131691923644350464_n.jpg',
 'images/5_33 106138130_10220534131076002_8014245700276487117_n.jpg',
 'images/10_0 74620824_10217973801189355_6879656113399136256_n.jpg',
 'images/15_01 22712585_10213751604877086_1112620062853664594_o.jpg',
 'images/10_0 69587723_10217531521412637_6714417760765476864_n.jpg',
 'images/12_1 50458285_10215922184780227_6800444615674560512_n.jpg',
 'images/NA 122906292_10221425212992493_986834629919058392_n.jpg',
 'images/10_0 66510121_10217119054581224_47845845483126784_n.jpg',
 'images/14_8 59022840_102166047730

In [55]:

image_path_list = glob(".cache/*")
image_path_list

['.cache/5_64 S__358637632.jpg',
 '.cache/11_45 4F8978FD-BEFD-49A2-B571-C989BD1AB234.jpg',
 '.cache/7_0 S__12075020.jpg',
 '.cache/11_0 S__12107796.jpg',
 '.cache/10_0 S__110002185.jpg',
 '.cache/25_0 S__259538957.jpg',
 '.cache/11_31 S__56500236.jpg',
 '.cache/3_0 S__56778755.jpg',
 '.cache/53_7 S__259457111.jpg',
 '.cache/10_0 S__259563536.jpg',
 '.cache/10_01 009EC4D6-3511-44F6-8621-106E78629E6C.jpg',
 '.cache/5_03 S__110616579.jpg']

In [19]:
def get_distance_from_path(image_path:str):
    image_file_name = image_path.split("/")[1]
    distance = image_file_name.split(" ")[0].replace("_",".")
    if distance == "NA":
        return -1
    else:
        return float(distance)



In [52]:
import re
def get_number_before_km(text):
    pattern = r"(\d+\.?\d*)\s*km"

    match = re.search(pattern, text)
    if match:
        number_before_km = match.group(1)
        return number_before_km
    return ""

text = "The distance is a 12.5 km."
print(get_number_before_km(text))
text = "The distance is a12.5 km."
print(get_number_before_km(text))
print(get_number_before_km("15.31 km"))

12.5
12.5
15.31


In [43]:
get_number_before_km(" 15.31 km")

''

# Pytesseract

In [1]:
from PIL import Image
import pytesseract

# Load the image from the file
image_path = 'images/garmin.jpg'  # Replace with your screenshot path
img = Image.open(image_path)

# Use pytesseract to do OCR on the image
extracted_text = pytesseract.image_to_string(img)

# Print the extracted text to see what was recognized
print(extracted_text)

# You can then filter out the distance using basic string operations
# Assuming the distance is in the format 'Distance: 5.0 km'
for line in extracted_text.splitlines():
    if 'km' in line.lower() or 'distance' in line.lower():
        print(f'Extracted Distance: {line}')


13:12 cose 2 ON Hl dull 73%

< Running Co:

Overview Stats Laps Charts Gear

Slower "MEE Faster

ee 3 Augi11@05:40 © vy ra

Bangkok Yai - Easy Run a

How did it go?

21.20:

Distance
150 bpm ® 5:16 skm ©
Avg Heart Rate Avg Pace
1:51:47 1,276
Total Time Total Calories

Extracted Distance: Distance
Extracted Distance: 150 bpm ® 5:16 skm ©


In [8]:
from PIL import Image
import pytesseract
import cv2

# Load the image from the file
image_path = './images/garmin.jpg'
img = cv2.imread(image_path)

# Convert to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Apply thresholding to make the text more distinct
_, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)

# Save the processed image for reference
processed_image_path = './images/processed_image.png'
cv2.imwrite(processed_image_path, thresh)

# Use pytesseract to do OCR on the processed image
extracted_text = pytesseract.image_to_string(Image.open(processed_image_path))

# Filter out the distance value
distance = None
for line in extracted_text.splitlines():
    if 'km' in line.lower():
        distance = line.strip()
        break

distance, extracted_text


(None,
 'B12 dc 99 « OAs. Heal 73%e\n\n€ Running a\n\nOverview Stats Laps Charts Gear\n\nSlower a Faster\n\n+ 3 Augii@o0540 @© ty ct\nBangkok Yai - Easy Run ,\nHow did it go? |\n\n21.20 in\n\nDistance\n\n150 bpm & 5:16 sm &\nAvg Heart Rate Avg Pace\n1:51:47 1,276\n\nTotal Time Total Calories\n')

# Cloud Vision API

In [ ]:
import base64
import json
import os


import functions_framework

from google.cloud import vision


vision_client = vision.ImageAnnotatorClient()


path = "images/garmin.jpg"
import io
with io.open(path, 'rb') as image_file:
        content = image_file.read()

image = vision.Image(content=content)
text_detection_response = vision_client.text_detection(image=image)

# EasyOCR

In [57]:
# Read text from an image
import easyocr
# Create an OCR reader object
reader = easyocr.Reader(['en'])
def get_distance_easyocr(image_path:str):
    result = reader.readtext(image_path)
    distance_list = []
    prev_text = ""
    if len(result)>1:
        prev_text = result[0][1].strip()
    for detection in result[1:]:
        text:str = detection[1]
        text = text.strip().lower()
        # if contain only km, the distance might be in previous text.
        if text == "km":
            text = prev_text + text
        
        if "km" in text and "/km" not in text and "km/" not in text:
            distance = get_number_before_km(text)
            try:
                distance = float(distance)
                distance_list.append(distance)
            except:
                print(text)
                print("not a float")
        prev_text = text
    if len(distance_list)==1:
        return distance_list[0]
    return -1

/Users/naveen/Library/Caches/pypoetry/virtualenvs/running-slip-reader-RFE_Jy1P-py3.11/lib/python3.11/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimenta

In [56]:
incorrect_count = 0
for image_path in image_path_list:
    distance = get_distance_easyocr(image_path)
    target = get_distance_from_path(image_path)

    if distance != target:
        incorrect_count+=1
        print(image_path)
        print(target)
        print(distance)
        print("============")

print(f"score = {len(image_path_list)-incorrect_count}/{len(image_path_list)}")



5'39" ikm
not a float
5'39" ikm
not a float
ikm
not a float
best km
not a float
11 bykm
not a float
7'49" gkm
not a float
.cache/3_0 S__56778755.jpg
3.0
-1
.cache/53_7 S__259457111.jpg
53.7
-1
7:30 ikm
not a float
ikm
not a float
.cache/10_01 009EC4D6-3511-44F6-8621-106E78629E6C.jpg
10.01
-1
score = 9/12


# Doctr

In [4]:
from doctr.models import ocr_predictor
from doctr.io import DocumentFile
# Load an image
image_path = 'images/garmin.jpg'

from doctr.io import DocumentFile
from doctr.models import ocr_predictor

model = ocr_predictor(pretrained=True)
# PDF
doc = DocumentFile.from_images(image_path)
# Analyze
result = model(doc)

/Users/naveen/Library/Caches/pypoetry/virtualenvs/running-slip-reader-RFE_Jy1P-py3.11/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/naveen/Library/Caches/pypoetry/virtualenvs/running-slip-reader-RFE_Jy1P-py3.11/lib/python3.11/site-packages/keras/src/layers/layer.py:1383: UserWarning: Layer 'fast_conv_layer_33' looks like it has unbuilt state, but Keras is not able to trace the layer `call()` in order to build it automatically. Possible causes:
1. The `call()` method of your layer may be crashing. Try to `__call__()` the layer eagerly on some test input first to see if it works. E.g. `x = np.random.random((3, 4)); y = layer(x)`
2. If the `call()` method is correct, then you may need to implement t

AttributeError: 'Sequential' object has no attribute 'get_output_at'

# Keras-OCR


In [5]:
import keras_ocr
pipeline = keras_ocr.pipeline.Pipeline()
images = ['images/garmin.jpg']  # List of image file paths
predictions = pipeline.recognize(images)
for image, prediction in zip(images, predictions):
    keras_ocr.tools.drawAnnotations(image=image, predictions=prediction)

Looking for /Users/naveen/.keras-ocr/craft_mlt_25k.h5


ValueError: Unrecognized keyword arguments passed to Dense: {'weights': [array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
   